In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import newton
from scipy.stats import *

In [2]:
def CIR(r0, alpha, beta, gamma, sigma, T, paths):
    np.random.seed(7)
    dt = 1/120
    steps = int(T/dt)
    rt = np.zeros((paths, steps + 1))
    rt[:, 0] = r0
    
    for i in range(1, steps):
        rt[:,i] = (np.maximum(rt[:,i-1], 0) + (alpha+beta*np.maximum(rt[:,i-1], 0))*dt 
                    + sigma*(np.maximum(rt[:,i-1], 0))**gamma * np.sqrt(dt)*np.random.normal(0, 1, paths))
    return rt

In [3]:
def discount_factor(r0, alpha, beta, gamma, sigma, T, paths):
    dt = 1/120
    ru = CIR(r0, alpha, beta, gamma, sigma, T, paths)
    df = np.exp(-np.cumsum(ru*dt , axis = 1))

    pay_time = np.arange(1, T*12 + 1) * 10 - 1
    
    return df[:, pay_time]

In [4]:
def Zero10Yr(rt, alpha, beta, gamma, sigma, T, t):
    
    h1 = np.sqrt(alpha**2 + 2*sigma**2)
    h2 = (alpha + h1)/2
    h3 = (2*(-alpha/beta))/sigma**2
    
    A = ((h1 * np.exp(h2*(T - t)))/(h2 * (np.exp(h1*(T - t)) - 1) + h1))**h3
    B = (np.exp(h1*(T - t)) - 1)/(h2 * (np.exp(h1*(T - t)) - 1) + h1)
    
    return A*np.exp(-B*rt)

In [5]:
def spotRate_10yr(r0, alpha, beta, gamma, sigma, T, paths):
   
    ru = CIR(r0, alpha, beta, gamma, sigma, T, paths)
    zeros10yr = np.array(list(map(lambda rt: Zero10Yr(rt, alpha, beta, gamma, sigma, 10, 0), ru)))
    spot_rate = -1/10*np.log(zeros10yr)
    
    pay_time = np.arange(1, T*12 + 1) * 10 - 1

    return spot_rate[:, pay_time]

In [6]:
def refinanceIncentive(R, r_120):
  RI = 0.28 + 0.14*np.arctan(-8.57 + 430*(R - r_120))
  return RI

In [7]:
def burnoutEffect(PV_pre, PV_0):
  BU = 0.3 + 0.7*(PV_pre/PV_0)
  return BU

In [8]:
def seasonality(t):
  SG = np.minimum(1, t/30)
  return SG

In [9]:
def seasoning(month):
  SY = [0.94, 0.76, 0.74, 0.95, 0.98, 0.92, 0.98, 1.10, 1.18, 1.22, 1.23, 0.98]
  return SY[month]

In [10]:
def CPR_t(t, R, r_120, PV_pre, PV_0):
  RI_t = refinanceIncentive(R, r_120)
  BU_t = burnoutEffect(PV_pre, PV_0)
  SG_t = seasonality(t)
  SY_t = seasoning(int((t - 1)%12))
  CPR_t = RI_t * BU_t * SG_t * SY_t
  return CPR_t

In [11]:
def Numerix_IO_PO(PV_0, total_years, WAC, r0, k, r_mean, sd):
   
    # initialize parameters
    r = WAC/12
    N = total_years*12
    paths = 10000
    PV_pre = np.array([PV_0]*paths)
    ct = np.zeros((paths, N))
    
    # Initialize PO, IO portion of the MBS
    Interest = np.zeros((paths, N))
    Principal = np.zeros((paths, N))
    
    # find discount factor
    dft = discount_factor(r0, sd, k, r_mean, total_years, paths)
    
    # find 10 year spot yield
    r_10 = spotRate_10yr(r0, sd, k, r_mean, total_years, paths)
     
    for t in range(1 , N + 1):
        # find CPRt
        CPRt_Numerix = CPR_t(t, WAC, r_10[:, t - 1], PV_pre, PV_0)
        # find cash flow at t
        ct[:, t - 1] = cash_flow_t(t, total_years, PV_pre, r, N, CPRt_Numerix)
        # find total principal payment
        Interest[:, t - 1] = PV_pre*r
        TPPt = ct[:, t - 1] - PV_pre*r
        Principal[:, t - 1] =  TPPt
        # Update next period present value, PVt
        PV_pre = PV_pre - TPPt

   
    IO = np.mean(sum(dft*Interest, axis = 1))
    PO = np.mean(sum(dft*Principal, axis = 1))
    
    return [IO, PO]

In [12]:
def cash_flow_t(t, T, PV_pre, r, N, CPR_t):
    
    MPt = (PV_pre*r)/(1 - (1 + r)**(-N + (t - 1)))

    SPPt = PV_pre*r*(1/(1 - (1 + r)**(-N + (t - 1))) -  1)
    
    PPt = (PV_pre - SPPt)*(1 - (1 - CPR_t)**(1/12))
    
    ct = MPt + PPt
    return ct